In [1]:
import gensim
import numpy as np
import nltk
import os
from nltk.corpus import stopwords

import pickle

stopword = stopwords.words('english')
stopword += ['?','!','.',',',':',';']

In [2]:
with open("all_handles.pkl", 'rb') as picklefile: 
    corpus = pickle.load(picklefile)

In [5]:
dictionary = gensim.corpora.Dictionary(corpus)

TypeError: decoding to str: need a bytes-like object, tuple found

In [3]:
type(corpus)

list

In [4]:
def chapter_reader():
    fname = 'corpus.txt'
    with open(fname, 'r') as f:
        for i in range(573936):
            txt = f.readline()
            yield (x for x in 
                gensim.utils.tokenize(txt, lowercase=True, deacc=True, 
                                      errors="ignore")
                if x not in stopword)
corpus = []
dictionary = gensim.corpora.Dictionary(chapter_reader())
dictionary.filter_extremes(no_below=1, no_above=0.8, keep_n=100000)
for values in chapter_reader():
    corpus.append(dictionary.doc2bow(values))

In [5]:
gensim.corpora.MmCorpus.serialize('handle_corpus.mm', corpus)
corpus = gensim.corpora.MmCorpus('handle_corpus.mm')

In [6]:
print(corpus[1450])

[(104, 1.0), (133, 1.0), (139, 1.0), (245, 1.0), (592, 1.0), (787, 1.0), (1111, 1.0), (1431, 1.0), (3005, 1.0), (3006, 1.0)]


In [24]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [7]:
for i,vector in enumerate(corpus):
    most_index, most_count = max(vector, key=lambda item: item[1])
    print( "tweet " + str(i+1) + " most used word: ",end='')
    print( dictionary[most_index], most_count)

tweet 1 most used word: record 1.0
tweet 2 most used word: report 1.0
tweet 3 most used word: report 1.0
tweet 4 most used word: tonight 1.0
tweet 5 most used word: tomorrow 1.0
tweet 6 most used word: breaking 1.0
tweet 7 most used word: report 1.0
tweet 8 most used word: congratulations 1.0
tweet 9 most used word: part 1.0
tweet 10 most used word: sunday 1.0
tweet 11 most used word: amp 1.0
tweet 12 most used word: video 1.0
tweet 13 most used word: amp 1.0
tweet 14 most used word: stage 1.0
tweet 15 most used word: ready 1.0
tweet 16 most used word: airs 1.0
tweet 17 most used word: spoke 1.0
tweet 18 most used word: lawyers 1.0
tweet 19 most used word: new 1.0
tweet 20 most used word: public 1.0
tweet 21 most used word: new 1.0
tweet 22 most used word: think 1.0
tweet 23 most used word: time 1.0
tweet 24 most used word: tonight 1.0
tweet 25 most used word: sunday 1.0
tweet 26 most used word: sunday 1.0
tweet 27 most used word: th 2.0
tweet 28 most used word: report 1.0
tweet 29 mos

ValueError: max() arg is an empty sequence

In [8]:
lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=5)
lsi.print_topics()

[(0,
  '0.939*"trump" + 0.118*"amp" + 0.081*"says" + 0.076*"president" + 0.069*"donald" + 0.057*"new" + 0.055*"w" + 0.051*"via" + 0.050*"clinton" + 0.046*"house"'),
 (1,
  '0.866*"amp" + -0.227*"trump" + 0.186*"w" + 0.113*"us" + 0.099*"new" + 0.094*"pm" + 0.093*"today" + 0.093*"et" + 0.068*"one" + 0.066*"live"'),
 (2,
  '-0.429*"amp" + 0.266*"one" + 0.261*"says" + 0.237*"house" + 0.233*"new" + 0.232*"us" + -0.206*"trump" + 0.189*"like" + 0.162*"white" + 0.155*"president"'),
 (3,
  '-0.606*"pm" + -0.542*"et" + -0.219*"live" + -0.216*"w" + 0.152*"says" + 0.137*"amp" + -0.127*"tonight" + 0.122*"like" + -0.120*"join" + 0.114*"one"'),
 (4,
  '-0.592*"says" + -0.422*"pres" + 0.289*"via" + 0.249*"like" + 0.213*"gt" + 0.176*"new" + -0.169*"health" + -0.163*"care" + -0.161*"bill" + 0.150*"one"')]

In [9]:
tfidf = gensim.models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]
lsi_tfidf = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
lsi_tfidf.print_topics()

[(0,
  '-1.000*"nan" + -0.002*"rally" + -0.002*"action" + -0.001*"saturday" + -0.001*"amp" + -0.001*"join" + -0.001*"convention" + -0.001*"wlib" + -0.001*"justice" + -0.001*"keynote"'),
 (1,
  '0.638*"thank" + 0.271*"trump" + 0.261*"thanks" + 0.140*"amp" + 0.101*"good" + 0.094*"president" + 0.094*"et" + 0.093*"one" + 0.093*"today" + 0.092*"pm"'),
 (2,
  '-0.736*"thank" + 0.300*"trump" + 0.126*"amp" + 0.111*"pm" + 0.102*"et" + 0.093*"w" + 0.091*"president" + 0.090*"via" + 0.089*"says" + 0.083*"house"'),
 (3,
  '-0.931*"thanks" + 0.195*"thank" + 0.135*"et" + 0.078*"cnn" + 0.077*"pm" + 0.077*"trump" + 0.071*"sunday" + -0.069*"watching" + 0.063*"p" + 0.060*"interview"'),
 (4,
  '-0.543*"pm" + -0.533*"et" + 0.263*"trump" + -0.233*"live" + -0.163*"w" + -0.150*"join" + -0.129*"tonight" + -0.117*"thanks" + -0.117*"tune" + 0.099*"gt"')]

In [10]:
lda = gensim.models.LdaModel(corpus, id2word=dictionary, num_topics=5)
lda.print_topics()

[(0,
  '0.021*"obama" + 0.009*"like" + 0.008*"good" + 0.007*"people" + 0.007*"get" + 0.007*"yes" + 0.006*"one" + 0.005*"better" + 0.005*"let" + 0.005*"oh"'),
 (1,
  '0.026*"trump" + 0.012*"gop" + 0.009*"via" + 0.009*"says" + 0.009*"house" + 0.008*"bill" + 0.006*"care" + 0.006*"health" + 0.006*"fox" + 0.005*"news"'),
 (2,
  '0.067*"trump" + 0.018*"via" + 0.014*"says" + 0.013*"president" + 0.013*"clinton" + 0.010*"donald" + 0.009*"campaign" + 0.009*"hillary" + 0.008*"russia" + 0.008*"house"'),
 (3,
  '0.020*"amp" + 0.014*"thanks" + 0.012*"thank" + 0.012*"great" + 0.012*"today" + 0.010*"week" + 0.010*"rt" + 0.009*"tonight" + 0.009*"new" + 0.008*"watch"'),
 (4,
  '0.010*"one" + 0.009*"like" + 0.008*"think" + 0.008*"would" + 0.007*"right" + 0.007*"know" + 0.006*"election" + 0.005*"comey" + 0.005*"people" + 0.005*"policy"')]